# Plot a RM

Load GitHub repository, requirements and libraries

In [6]:
# download ACHP graph_mixer
!rm cultural-heritage -R
!git clone https://github.com/achp-project/cultural-heritage.git
!pip install  -q -r  cultural-heritage/graph-parser/requirements.txt
%cd /content/cultural-heritage/graph-parser/
import graph_mixer as gm

# download EAMENA erms
%cd /content/
!rm eamena-functions -R
!git clone https://github.com/eamena-project/eamena-functions.git
%cd /content/eamena-functions/erms
import erms

# librairies
import os
import urllib.request
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact, interactive
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import re
from IPython.display import HTML
from pyvis import network as net
import folium
import json

rm: cannot remove 'cultural-heritage': No such file or directory
Cloning into 'cultural-heritage'...
remote: Enumerating objects: 3009, done.
remote: Counting objects: 100% (277/277), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 3009 (delta 183), reused 269 (delta 181), pack-reused 2732
Receiving objects: 100% (3009/3009), 29.40 MiB | 24.01 MiB/s, done.
Resolving deltas: 100% (2217/2217), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.0/157.0 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.3/199.3 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.3/300.3 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 72.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.

Load libraries

List of available RMs

In [11]:
%cd /content/cultural-heritage/graph-parser/
!rm inputResourceModels -R
!mkdir -p inputResourceModels
remote_source_files = gm.rm_list()
remote_source_files

/content/cultural-heritage/graph-parser
rm: cannot remove 'inputResourceModels': No such file or directory


{'MAPSS': 'https://raw.githubusercontent.com/achp-project/prj-mapss/main/pkg/graphs/Heritage%20Place%20(3).json',
 'MAHS': 'https://raw.githubusercontent.com/achp-project/prj-mahs/main/Site.json',
 'MAHSA': 'https://raw.githubusercontent.com/achp-project/prj-mahsa/main/resource-models/Heritage%20Location%20Resource%20Model.json',
 'MAEASAM': 'https://raw.githubusercontent.com/achp-project/prj-maeasam/main/Site.json',
 'EAMENA': 'https://raw.githubusercontent.com/achp-project/prj-eamena-marea/main/resource_models/Heritage%20Place.json'}

Select one RM to plot by its key (ex: `MAPSS`, `MAHS`, etc.)

In [12]:
gm.rm_one_selected('EAMENA', remote_source_files)

Create subgraph and comparison dataframe csv files by running [graph-parser.py](https://github.com/achp-project/cultural-heritage/blob/main/graph-parser/graph_parser.py) and [graph-comparator.py](https://github.com/achp-project/cultural-heritage/blob/main/graph-parser/graph_comparator.py)

In [13]:
!python /content/cultural-heritage/graph-parser/graph_comparator.py -d -m list -o subgraphMetrics.csv inputResourceModels/*.json
subgraph_metrics = pd.read_csv('subgraphMetrics.csv')
subgraph_metrics

,Unnamed: 0,graph_name,graph_id,source_property,target_property,relation_type,source_id,target_id,source_name,target_name
0,0,EAMENA_Heritage%20Place,34cfe98e-c2c0-11ea-9026-02e7594ce0a0,E27_Site,E53_Place,P53_has_former_or_current_location,34cfea3b-c2c0-11ea-9026-02e7594ce0a0,34cfe9b6-c2c0-11ea-9026-02e7594ce0a0,Heritage Place,Geography
1,1,EAMENA_Heritage%20Place,34cfe98e-c2c0-11ea-9026-02e7594ce0a0,E27_Site,E53_Place,P53_has_former_or_current_location,34cfea3b-c2c0-11ea-9026-02e7594ce0a0,3080eebe-c2c5-11ea-9026-02e7594ce0a0,Heritage Place,Geometry
2,2,EAMENA_Heritage%20Place,34cfe98e-c2c0-11ea-9026-02e7594ce0a0,E54_Dimension,E60_Number,P90_has_value,34cfea1f-c2c0-11ea-9026-02e7594ce0a0,34cfea40-c2c0-11ea-9026-02e7594ce0a0,Depth/Elevation,Minimum Depth/Max Elevation
3,3,EAMENA_Heritage%20Place,34cfe98e-c2c0-11ea-9026-02e7594ce0a0,E54_Dimension,E60_Number,P90_has_value,34cfea1f-c2c0-11ea-9026-02e7594ce0a0,34cfea42-c2c0-11ea-9026-02e7594ce0a0,Depth/Elevation,Maximum Depth/Min Elevation
4,4,EAMENA_Heritage%20Place,34cfe98e-c2c0-11ea-9026-02e7594ce0a0,E54_Dimension,E60_Number,P90_has_value,34cfea8c-c2c0-11ea-9026-02e7594ce0a0,34cfea47-c2c0-11ea-9026-02e7594ce0a0,Dimension,Measurement Number
...,...,...,...,...,...,...,...,...,...,...
154,154,EAMENA_Heritage%20Place,34cfe98e-c2c0-11ea-9026-02e7594ce0a0,E13_Attribute_Assignment,S4_Observation,P17_was_motivated_by,34cfe99e-c2c0-11ea-9026-02e7594ce0a0,34cfe9ce-c2c0-11ea-9026-02e7594ce0a0,Disturbance Cause Assignment,Damage Observation
155,155,EAMENA_Heritage%20Place,34cfe98e-c2c0-11ea-9026-02e7594ce0a0,S5_Inference_Making,E39_Actor,P14_carried_out_by,34cfe9fb-c2c0-11ea-9026-02e7594ce0a0,34cfea54-c2c0-11ea-9026-02e7594ce0a0,Threat Inference Making,Threat Inference Making Assessor
156,156,EAMENA_Heritage%20Place,34cfe98e-c2c0-11ea-9026-02e7594ce0a0,I2_Belief,I2_Belief,P117i_includes,38cff734-c77b-11ea-a292-02e7594ce0a0,38cff731-c77b-11ea-a292-02e7594ce0a0,Cultural Period Belief,Cultural Sub-period Belief
157,157,EAMENA_Heritage%20Place,34cfe98e-c2c0-11ea-9026-02e7594ce0a0,I2_Belief,E13_Attribute_Assignment,P117i_includes,34cfe9d4-c2c0-11ea-9026-02e7594ce0a0,34cfe9d7-c2c0-11ea-9026-02e7594ce0a0,Recommendation Plan,Priority Assignment


Show RM

In [39]:
rm_project= 'EAMENA'
erms_template_df = erms.erms_template()
erms_template_df['Enhanced record minimum standard'] = erms_template_df['Enhanced record minimum standard'].str.contains(r'Yes', case = False, na = False, regex = True).astype(int)
erms_template_df = erms_template_df.loc[erms_template_df['Enhanced record minimum standard'] == 1]
in_erms = list(erms_template_df['uuid_sql'])
df_color = erms.erms_field_colors()
G = gm.create_rm_graph(rm_project= rm_project, color_fields = df_color)
gm.plot_net_graph(G)

YA
